In [2]:
from yt_dlp import YoutubeDL
import os
import demucs.separate
import shlex
import pandas as pd
from difflib import SequenceMatcher
import webbrowser

from ytmusicapi import YTMusic
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
#Get your Client ID and Secret by creating an app:
webbrowser.open('https://developer.spotify.com/dashboard')

SPOTIPY_CLIENT_ID = ''
SPOTIPY_CLIENT_SECRET = ''

# # Optional: If you run out of YTMusic API tokens, authenticate again
# os.system("ytmusicapi oauth")

In [ ]:
if SPOTIPY_CLIENT_ID == '' or SPOTIPY_CLIENT_SECRET == '':
    raise Exception('Need a Client ID and Client Secret to continue')

PORT_NUMBER = 8080
SPOTIPY_REDIRECT_URI = 'http://localhost:'+str(PORT_NUMBER)
SCOPE = 'user-library-read'
CACHE = '.spotipyoauthcache'

auth_manager = SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                             client_secret=SPOTIPY_CLIENT_SECRET,
                             redirect_uri=SPOTIPY_REDIRECT_URI,
                             scope=SCOPE,
                             )

yt = YTMusic("oauth.json") if os.path.exists("oauth.json") else YTMusic()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
playlist_link = 'https://open.spotify.com/playlist/3q99oCvea638VTFr9jFnm5?si=404c04e63de445bb'

download_path = 'downloads'

model = 'htdemucs' # Options: 'htdemucs' 'htdemucs_ft' 'htdemucs_6s'

duration_threshold = 8

limit = 30

In [ ]:
def get_sp_songs_playlist(playlist_link):    
    playlist = sp.playlist(playlist_link)
    sp_songs = []
    for i in range(len(playlist['tracks']['items'])):
        title = playlist['tracks']['items'][i]['track']['name']
        artists = [artist['name'] for artist in playlist['tracks']['items'][i]['track']['artists']]
        duration = int(playlist['tracks']['items'][i]['track']['duration_ms']/1000)

        sp_songs.append({'title': title, 'artists':artists, 'duration_seconds': duration})
    return sp_songs

def get_yt_results(sp_song, duration_threshold = duration_threshold):
    search_song = sp_song['title']+' - ' + ', '.join(sp_song['artists']) # 'title - artist1, artist2, ...'
    yt_results = pd.DataFrame.from_dict(yt.search(search_song, filter='songs'))
    yt_results = yt_results.loc[:,['title', 'artists', 'duration_seconds', 'videoId']]
    return yt_results.loc[abs(yt_results['duration_seconds'] - sp_song['duration_seconds']) < duration_threshold]

def get_artist_scores(sp_song, yt_results):
    yt_artists_list = []
    for artist_dic in yt_results['artists']:
        yt_artists_list.append([artist['name'] for artist in artist_dic])
    sp_artists = sp_song['artists']
    artist_scores = []
    for yt_artists in yt_artists_list:
        artist_score = 0
        for yt_artist in yt_artists:
            for sp_artist in sp_artists:
                matching = SequenceMatcher(None, sp_artist.lower(), yt_artist.lower()).ratio()
                if matching >= 0.5:
                    artist_score += matching
        artist_scores.append(artist_score/len(sp_artists))
    return artist_scores

In [ ]:
def convertSpotifyToYouTube(sp_songs):
    links = []
    for i in range(0,len(sp_songs)):
        sp_song = sp_songs[i]
        yt_results = get_yt_results(sp_song, duration_threshold)
        if len(yt_results.index) > 0:
            artist_scores = get_artist_scores(sp_song, yt_results)
            best_result_Id = [x for _, x in sorted(zip(artist_scores, yt_results['videoId']), key=lambda pair: -pair[0])][0]
            links.append('https://www.youtube.com/watch?v='+best_result_Id)
        else:
            print('cannot find good results for '+ sp_song['title'])
    return links

In [ ]:
def linkName(link, ydl_opts):
  with YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download = False)
    return info_dict['title']

def download_audio(link, ydl_opts):
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(link)    

In [ ]:
def downloadYT(playlistPath = '', links = []):
    #Directory Create
    if playlistPath != '':
        if not os.path.exists(playlistPath):
            os.makedirs(playlistPath)

    #YTDownload Options
    ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': playlistPath+'/%(title)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',
    }],
    }

    #Download Links
    for link in links:
        #Only Download what's not already downloaded
        name = linkName(link, ydl_opts)
        if name+'.mp3' not in os.listdir(playlistPath):
            print(0,name)
            download_audio(link, ydl_opts)
        else:
            print(1,name)

def separateDirectoryDemucs(playlistPath = '', model = 'htdemucs'):
    #Directory Create  
    if playlistPath != '':
        if not os.path.exists(playlistPath+'/'+model):
            os.makedirs(playlistPath+'/'+model)
    
    #Demucs Separate what's not already separated
    for file in os.listdir(playlistPath):
        if os.path.splitext(file)[1] == '.mp3':
            if os.path.splitext(file)[0] not in os.listdir(playlistPath+'/'+model):
                demucs_params = '--mp3' +  (' "%s"' % (playlistPath+'/'+file)) + (' -o "%s"' % playlistPath) + (' -n ' + model)
                demucs.separate.main(shlex.split(demucs_params))
            else:
                print(1,file)

In [ ]:
sp_songs = get_sp_songs_playlist(playlist_link)
links = convertSpotifyToYouTube(sp_songs)
links

In [ ]:
# downloadYT(download_path, links)
# separateDirectoryDemucs(download_path, model)